# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Loading data into new dataframe
city_data_df = pd.read_csv("output_data/cities.csv")

#Removing the extraneous index column
city_data_df = city_data_df.drop(columns = "Unnamed: 0")

city_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Philippe,-21.3585,55.7679,69.35,78,26,9.69,RE,1633665460
1,Qaanaaq,77.4840,-69.3632,22.30,66,100,6.15,GL,1633665462
2,Vanimo,-2.6741,141.3028,87.67,62,34,9.46,PG,1633665463
3,Gamba,-2.6500,10.0000,75.69,88,100,4.90,GA,1633665464
4,Puerto Ayora,-0.7393,-90.3518,68.56,95,93,8.01,EC,1633665466
...,...,...,...,...,...,...,...,...,...
550,Lobanovo,57.8595,56.3026,33.71,86,0,11.18,RU,1633666305
551,Barsovo,61.1667,73.1667,41.13,85,100,11.43,RU,1633666307
552,Dandong,40.1292,124.3947,65.14,70,69,4.07,CN,1633666308
553,Kwinana,-32.2500,115.7667,67.30,63,95,12.39,AU,1633666310


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = city_data_df[["Lat", "Lng"]]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=city_data_df["Humidity"], 
                                 dissipating=False, max_intensity=None,
                                 point_radius=5)


# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Dropping any rows with null values

trimmed_cities_df = city_data_df.dropna(axis=0)
trimmed_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Philippe,-21.3585,55.7679,69.35,78,26,9.69,RE,1633665460
1,Qaanaaq,77.4840,-69.3632,22.30,66,100,6.15,GL,1633665462
2,Vanimo,-2.6741,141.3028,87.67,62,34,9.46,PG,1633665463
3,Gamba,-2.6500,10.0000,75.69,88,100,4.90,GA,1633665464
4,Puerto Ayora,-0.7393,-90.3518,68.56,95,93,8.01,EC,1633665466
...,...,...,...,...,...,...,...,...,...
550,Lobanovo,57.8595,56.3026,33.71,86,0,11.18,RU,1633666305
551,Barsovo,61.1667,73.1667,41.13,85,100,11.43,RU,1633666307
552,Dandong,40.1292,124.3947,65.14,70,69,4.07,CN,1633666308
553,Kwinana,-32.2500,115.7667,67.30,63,95,12.39,AU,1633666310


In [41]:
# Setting favored weather conditions for cities:
#Max temperature between 50 and 75 degrees, cloudiness between 20 and 80, and humidity less than 60.

ideal_cities_df = pd.DataFrame(trimmed_cities_df.loc[(trimmed_cities_df["Max Temp"] < 75) & (trimmed_cities_df["Max Temp"] > 50) \
                                       & (trimmed_cities_df["Cloudiness"] > 20) & (trimmed_cities_df["Cloudiness"] < 80) \
                                       & (trimmed_cities_df["Humidity"] < 60)])

ideal_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
96,Hobart,-42.8794,147.3294,67.86,49,75,9.22,AU,1633665605
119,Fortuna,40.5982,-124.1573,52.93,51,36,1.99,US,1633665653
149,Mount Gambier,-37.8333,140.7667,64.31,48,75,20.71,AU,1633665698
151,Tavira,37.1273,-7.6486,66.07,53,51,5.12,PT,1633665684
199,Petropavlovsk-Kamchatskiy,53.0452,158.6483,50.59,32,40,15.66,RU,1633665773
205,Dubbo,-32.2500,148.6167,73.53,28,56,11.50,AU,1633665782
290,Cedar City,37.6775,-113.0619,57.40,59,42,7.34,US,1633665655
375,Santa Cruz de la Sierra,-17.8000,-63.1667,73.74,58,33,3.89,BO,1633665795
431,San Luis,-33.2950,-66.3356,61.14,47,43,5.10,AR,1633666126
447,San Miguel de Tucumán,-26.8241,-65.2226,64.51,54,22,2.89,AR,1633666149


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
#Establish empty list of hotels
hotels = []

In [43]:
# find the closest hotel to coordinates

#establish URL and parameters

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in ideal_cities_df.iterrows():

    # get coordinates from df
    coords = str(row["Lat"]) + "," + str(row["Lng"])

    # add coordinates to params dict
    params['location'] = coords
    
    # assemble url and make API request
    cityName = row["City"]
    print(f"Retrieving Results for Index {index}: {cityName}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        #add each hotel to hotels list
        hotels.append(results[0]['name'])

        #error handling
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 96: Hobart.
Closest hotel is Hobart Magistrates Court.
Retrieving Results for Index 119: Fortuna.
Closest hotel is Kim Quintal- Real Estate Agent.
Retrieving Results for Index 149: Mount Gambier.
Closest hotel is Music Technology Hub - Day Options.
Retrieving Results for Index 151: Tavira.
Closest hotel is Pedro Lucas - Consultoria e Gestão Imobiliária.
Retrieving Results for Index 199: Petropavlovsk-Kamchatskiy.
Closest hotel is Tupichok.
Retrieving Results for Index 205: Dubbo.
Closest hotel is Hampden St before Wingewarra St.
Retrieving Results for Index 290: Cedar City.
Closest hotel is Cedar Canyon Nature Park.
Retrieving Results for Index 375: Santa Cruz de la Sierra.
Closest hotel is Milla 33.
Retrieving Results for Index 431: San Luis.
Closest hotel is Estudio Maldonado Paone & Asociados.
Retrieving Results for Index 447: San Miguel de Tucumán.
Closest hotel is Marmoleria Berral.


In [45]:
ideal_cities_df["Hotel Name"] = hotels

hotel_df = pd.DataFrame(ideal_cities_df)

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
96,Hobart,-42.8794,147.3294,67.86,49,75,9.22,AU,1633665605,Hobart Magistrates Court
119,Fortuna,40.5982,-124.1573,52.93,51,36,1.99,US,1633665653,Kim Quintal- Real Estate Agent
149,Mount Gambier,-37.8333,140.7667,64.31,48,75,20.71,AU,1633665698,Music Technology Hub - Day Options
151,Tavira,37.1273,-7.6486,66.07,53,51,5.12,PT,1633665684,Pedro Lucas - Consultoria e Gestão Imobiliária
199,Petropavlovsk-Kamchatskiy,53.0452,158.6483,50.59,32,40,15.66,RU,1633665773,Tupichok
205,Dubbo,-32.2500,148.6167,73.53,28,56,11.50,AU,1633665782,Hampden St before Wingewarra St
290,Cedar City,37.6775,-113.0619,57.40,59,42,7.34,US,1633665655,Cedar Canyon Nature Park
375,Santa Cruz de la Sierra,-17.8000,-63.1667,73.74,58,33,3.89,BO,1633665795,Milla 33
431,San Luis,-33.2950,-66.3356,61.14,47,43,5.10,AR,1633666126,Estudio Maldonado Paone & Asociados
447,San Miguel de Tucumán,-26.8241,-65.2226,64.51,54,22,2.89,AR,1633666149,Marmoleria Berral


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map



# Display figure
